In [1]:
import os
import pandas as pd
from collections import defaultdict
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path
from textwrap import dedent
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
import torch as tf
from collections import Counter
from datetime import datetime
import math
from sklearn.metrics import confusion_matrix
from tools import load_sensor_data_without_h, sample_sensor_data, manual_lable_array_list, most_frequent, extract_sensor_data, get_sensor_data, sample_data, sample_label, combine_to_one
from model import Classifier_dh

In [2]:
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
actor_1 = [1,2,3,4,5,6,7,8,9,10]
actor_2 = [11,12,13,14,31,32,33,34,35,36]
actor_3 = [15,16,17,18,37,38,39,40,41,42]
actor_4 = [43,44,45,46,47,48,49,50,51,52]
actor_5 = [53,54,55,56,57,58,59,60,61,62]
actor_6 = [63,64,65,66,67,68,69,70,71,72]
actor_7 = [76,77,78,79,80,81,82,83,84,85]
actor_8 = [86,87,88,89,90,91,92,93,94,95]
actor_9 = [97,98,99,100,101,102,103,104,105,106]
actor_10 = [107,108,109,110,111,112,113,114,115,116]
actor_11 = [117,118,119,120,121,122,123,124,125,126]
actor_12 = [128,129,130,131,132,133,134,135,136,137]
actor_13 = [138,139,140,141,142,143,144,145,146,147]
actor_14 = [148,149,150,151,152,153,154,155,156,157]
actor_15 = [161,162,163,164,165,166,167,168,169,170]
actor_16 = [171,172,173,174,175,176,177,178,179,180]
actor_17 = [181,182,183,184,185,186,187,188,189,190]
actor_18 = [191,192,193,194,195,196,197,198,199,200]
actor_19 = [201,202,203,204,205,206,207,208,209,210]
actor_20 = [212,213,214,215,216,217,218,219,220,221]
actor_21 = [223,224,225,226,227,228,229,230,231,232]
actor_22 = [233,234,235,236,237,238,239,240,241,242]
actor_23 = [243,244,245,246,247,248,249,250,251,252]
actor_24 = [254,255,256,257,258,259,260,261,262,263]
actor_25 = [264,265,266,267,268,269,270,271,272,273]
combine_list = actor_1 + actor_2 + actor_3 + actor_4 + actor_5 + actor_6 + actor_7 + actor_8 + actor_9 + actor_10+ actor_11 + actor_12 + actor_13 + actor_14 + actor_15+ actor_16 + actor_17 + actor_18 + actor_19 + actor_20+ actor_21 + actor_22 + actor_23 + actor_24 + actor_25
window_sz = 128
sample_sz = 128
lr = 0.001
n_epochs = 500
num_classes = 6
patience, trials = 100, 0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss(reduction='sum')
bs = 256

In [3]:
#manual_lable_array_list('../manual_lable.csv', combine_list)
#extract_sensor_data(combine_list) 

#label_list = np.load('../save_data/label_data/label_list.npy', allow_pickle=True).item()
#sensor_data = get_sensor_data('../save_data/original_data/')

#sample_data(sensor_data, combine_list, window_sz = window_sz, sample_sz = sample_sz)
#sample_label(label_list, combine_list, window_sz = window_sz, sample_sz = sample_sz)
   
sample_sensor_data = get_sensor_data('../save_data/sample_data/')
sample_sensor_label = get_sensor_data('../save_data/sample_label/')

In [4]:
sample_sensor_data[1].shape

(281, 39, 128)

In [5]:
bestacc = []
for actor in range(16,25):
    total_num_list = total_num_list = actor_1 + actor_2 + actor_3 + actor_4+actor_5+actor_6+actor_7+actor_8+actor_9+actor_10+actor_11+actor_12+actor_13+actor_14+actor_15+actor_16+actor_17+actor_18+actor_19+actor_20+actor_21+actor_22+actor_23+actor_24+actor_25
    val_num_list = total_num_list[actor*10 : actor*10+10]
    del total_num_list[actor*10 : actor*10+10]
    train_num_list = total_num_list
    train_sensor_data = combine_to_one(sample_sensor_data,train_num_list)
    val_sensor_data = combine_to_one(sample_sensor_data,val_num_list)
    train_sensor_label = combine_to_one(sample_sensor_label,train_num_list)
    val_sensor_label = combine_to_one(sample_sensor_label,val_num_list)
    model = Classifier_dh(train_sensor_data.shape[1], train_sensor_data.shape[2], num_classes).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    train_ds = TensorDataset(torch.tensor(train_sensor_data).float(), torch.tensor(train_sensor_label).long())
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=0)
    val_ds = TensorDataset(torch.tensor(val_sensor_data).float(), torch.tensor(val_sensor_label).long())
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)
    best_acc = 0
    loss_history = []
    acc_history = []
    print('Start model training')
    for epoch in range(1, n_epochs + 1):
        model.train()
        epoch_loss = 0
        for i, batch in enumerate(train_dl):
            x_raw, y_batch = [t.to(device) for t in batch]
            y_batch = tf.squeeze(y_batch)
            opt.zero_grad()
            out = model(x_raw)
            loss = criterion(out, y_batch)
            epoch_loss += loss.item()
            loss.backward()
            opt.step()
        loss_history.append(epoch_loss)
        model.eval()
        correct, total = 0, 0
        for batch in val_dl:
            x_raw, y_batch = [t.to(device) for t in batch]
            y_batch = tf.squeeze(y_batch)
            out = model(x_raw)
            preds = F.log_softmax(out, dim=1).argmax(dim=1)
            if preds.size()[0] > 1:
                total += y_batch.size(0)
                correct += (preds == y_batch).sum().item()
        acc = correct / total
        acc_history.append(acc)
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        if acc > best_acc:
            trials = 0
            best_acc = acc
            torch.save(model.state_dict(), 'best.pth')
            print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
        else:
            trials += 1
            if trials >= patience:
                print(f'Early stopping on epoch {epoch}')
                break
    bestacc.append(best_acc)

Start model training
Epoch 1 best model saved with accuracy: 32.31%
Epoch 2 best model saved with accuracy: 42.74%
Epoch 3 best model saved with accuracy: 46.69%
Epoch 4 best model saved with accuracy: 48.79%
Epoch 5 best model saved with accuracy: 50.57%
Epoch 6 best model saved with accuracy: 52.53%
Epoch 7 best model saved with accuracy: 53.38%
Epoch 8 best model saved with accuracy: 54.02%
Epoch:  10. Loss: 53772.9069. Acc.: 54.91%
Epoch 10 best model saved with accuracy: 54.91%
Epoch:  20. Loss: 42825.2482. Acc.: 47.94%
Epoch:  30. Loss: 37279.3787. Acc.: 46.62%
Epoch:  40. Loss: 35431.7553. Acc.: 46.98%
Epoch:  50. Loss: 33709.7432. Acc.: 44.38%
Epoch:  60. Loss: 32936.2518. Acc.: 42.92%
Epoch:  70. Loss: 32223.0749. Acc.: 44.52%
Epoch:  80. Loss: 31199.7177. Acc.: 48.58%
Epoch:  90. Loss: 30723.5865. Acc.: 46.87%
Epoch: 100. Loss: 30602.3506. Acc.: 47.58%
Epoch: 110. Loss: 30072.1980. Acc.: 47.65%
Early stopping on epoch 110
Start model training
Epoch 1 best model saved with acc

Epoch:  60. Loss: 32729.7295. Acc.: 49.75%
Epoch 60 best model saved with accuracy: 49.75%
Epoch:  70. Loss: 31915.4401. Acc.: 50.57%
Epoch 70 best model saved with accuracy: 50.57%
Epoch 75 best model saved with accuracy: 51.35%
Epoch:  80. Loss: 30771.5353. Acc.: 47.01%
Epoch:  90. Loss: 30344.8494. Acc.: 46.83%
Epoch: 100. Loss: 29922.1380. Acc.: 48.01%
Epoch: 110. Loss: 29388.3551. Acc.: 44.38%
Epoch: 120. Loss: 29318.1729. Acc.: 45.41%
Epoch: 130. Loss: 29143.9308. Acc.: 47.37%
Epoch: 140. Loss: 28703.8739. Acc.: 47.58%
Epoch: 150. Loss: 28694.9453. Acc.: 43.91%
Epoch: 160. Loss: 28230.9795. Acc.: 42.53%
Epoch: 170. Loss: 28069.0202. Acc.: 46.69%
Early stopping on epoch 175
Start model training
Epoch 1 best model saved with accuracy: 29.07%
Epoch 2 best model saved with accuracy: 34.84%
Epoch:  10. Loss: 51732.3281. Acc.: 34.77%
Epoch 11 best model saved with accuracy: 35.55%
Epoch 14 best model saved with accuracy: 37.72%
Epoch:  20. Loss: 41852.9562. Acc.: 34.38%
Epoch:  30. Los

In [6]:
bestacc

[0.5491103202846975,
 0.7654804270462633,
 0.4804270462633452,
 0.6238434163701068,
 0.5380782918149466,
 0.48825622775800714,
 0.45516014234875446,
 0.5135231316725979,
 0.37722419928825623]

In [ ]:
A=[39.86, 54.80, 67.65, 44.23, 69.04, 65.87, 70.82, 53.27, 51.71, 69.40, 68.90, 59.82, 56.69, 69.07, 62.38, 60.32, 56.98, 73.74, 51.07, 64.84, 58.97, 51.25, 48.26, 52.63, 37.4.0.0007]

In [ ]:
A=[44.98, 41.00, 66.73, 45.05, 72.85, 66.48, 75.37, 54.20, 53.67, 69.50, 67.97, 52.28, 60.71, 63.99, 60.50, 57.37]

In [ ]:
sum(A)/25

In [ ]:
A=[54.80, 67.65, 44.23, 69.04, 65.87, 70.82, 53.27, 51.71, 69.40, 68.90, 59.82, 56.69, 69.07, 62.38, 60.32, 56.98, 73.74, 51.07, 64.84, 58.97, 51.25, 48.26, 52.63]

In [ ]:
sum(A)/25